In [11]:
from pyspark.sql import SparkSession

appName = "PySpark Example"
master = "local"
# Create Spark session
spark = SparkSession.builder \
    .appName(appName) \
    .master(master) \
    .config("spark.jars", "/usr/local/spark/jars/mysql-connector-java-8.0.30.jar") \
    .getOrCreate()



In [12]:

server = "localhost"
port = 3306
database = "nation"
jdbc_url = f"jdbc:mysql://{server}:{port}/{database}?permitMysqlScheme"


user = "alinizar"
password = "alinizarpwd"
jdbc_driver = "com.mysql.cj.jdbc.Driver"


properties = {
    "user": user,
    "password": password,
    "driver": jdbc_driver
}


In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [14]:


countries_df = spark.read.jdbc(jdbc_url, "(select * from countries) tab", properties=properties)
country_stats_df = spark.read.jdbc(jdbc_url, "(select * from country_stats) tab", properties=properties)


In [15]:

# Create dataframe countries with stats
countries_df1 = (
country_stats_df
    .join(countries_df, country_stats_df.country_id == countries_df.country_id, "left"))

In [16]:
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression


In [18]:

# Linear Regression to predict GDP based on population and area
assembler = VectorAssembler(inputCols=["population", "area"], outputCol="features")
regression_data = assembler.transform(countries_df1)
regression_model = LinearRegression(featuresCol="features", labelCol="gdp").fit(regression_data)

# Display the coefficients of the linear regression model
print("Coefficients: ", regression_model.coefficients)
print("Intercept: ", regression_model.intercept)

# Prediction: Add GDP predictions to the country_stats DataFrame
predictions_df = regression_model.transform(regression_data)

# Additional Analysis 1: Show countries with the largest positive GDP prediction residuals
residuals_df = predictions_df.withColumn("residual", col("gdp") - col("prediction"))
largest_residuals = residuals_df.orderBy("residual", ascending=False).limit(5)
largest_residuals.show()

# working with SparkSQL, select columns 
largest_residuals.createOrReplaceTempView("my_table")
result_df_sql=spark.sql("select name,year,population,gdp,area,prediction,residual from my_table")
result_df_sql.show()


24/06/07 18:03:21 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-93fdc6ee-cebe-47f9-9195-aece353d728b/2f/temp_shuffle_99247e80-e149-4cc0-b511-152b8695ded3
24/06/07 18:03:21 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-93fdc6ee-cebe-47f9-9195-aece353d728b/36/temp_shuffle_01a37f12-db71-4bbf-a7b9-60eaa76fae0e
24/06/07 18:03:21 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-93fdc6ee-cebe-47f9-9195-aece353d728b/38/temp_shuffle_0b07222f-9794-46c3-abee-b4dc6aff66b8
24/06/07 18:03:21 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-93fdc6ee-cebe-47f9-9195-aece353d728b/37/temp_shuffle_e706782a-c1a1-45da-ac06-b9ce3a1a3785
24/06/07 18:03:21 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-93fdc6ee-cebe-47f9-9195-aece353d728b/2d/temp_shuffle_bebbc5c4-5e33-4e74-b4f1-d5c260d0ace2
24/06/07 18:03:21 WARN DiskBlockObjectWriter: Error deleting /tmp/blockmgr-93fdc6ee-cebe-47f9-9195-aece353d728b/33/temp_shuffle_0aec2735-9e67-4d77-9f1d-54c6a459ac80
24/06/07 1

Py4JJavaError: An error occurred while calling o191.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5.0 (TID 5) (myvm.internal.cloudapp.net executor driver): java.sql.SQLException: DAY_OF_MONTH
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:129)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:97)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:89)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:63)
	at com.mysql.cj.jdbc.exceptions.SQLError.createSQLException(SQLError.java:73)
	at com.mysql.cj.jdbc.exceptions.SQLExceptionsMapping.translateException(SQLExceptionsMapping.java:85)
	at com.mysql.cj.jdbc.result.ResultSetImpl.getDate(ResultSetImpl.java:784)
	at org.apache.spark.sql.execution.datasources.jdbc.JdbcUtils$.$anonfun$makeGetter$2(JdbcUtils.scala:398)
	at org.apache.spark